In [12]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from IPython.display import display

from qiskit import QuantumCircuit
from qiskit.compiler import transpile
from qiskit_aer import Aer
from qiskit_aer import AerSimulator

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library import QFTGate, UnitaryGate, RYGate, XGate, GlobalPhaseGate
from qiskit.visualization import plot_histogram

In [13]:
from random import randint
import os
import matplotlib.pyplot as plt
import numpy as np

#Ship Class
class Ship:
    def __init__(self, size, orientation, location):
        self.size = size

        if orientation == 'horizontal' or orientation == 'vertical':
            self.orientation = orientation
        else:
            raise ValueError("Value must be 'horizontal' or 'vertical'.")

        if orientation == 'horizontal':
            if location['row'] in range(row_size):
                self.coordinates = []
                for index in range(size):
                    if location['col'] + index in range(col_size):
                        self.coordinates.append({'row': location['row'], 'col': location['col'] + index})
                    else:
                        raise IndexError("Column is out of range.")
            else:
                raise IndexError("Row is out of range.")
        elif orientation == 'vertical':
            if location['col'] in range(col_size):
                self.coordinates = []
                for index in range(size):
                    if location['row'] + index in range(row_size):
                        self.coordinates.append({'row': location['row'] + index, 'col': location['col']})
                    else:
                        raise IndexError("Row is out of range.")
            else:
                raise IndexError("Column is out of range.")

        if self.filled():
            print_board(board)
            print(" ".join(str(coords) for coords in self.coordinates))
            raise IndexError("A ship already occupies that space.")
        else:
            self.fillBoard()

    def filled(self):
        for coords in self.coordinates:
            if board[coords['row']][coords['col']] == 1:
                return True
        return False

    def fillBoard(self):
        for coords in self.coordinates:
            board[coords['row']][coords['col']] = 1

    def contains(self, location):
        for coords in self.coordinates:
            if coords == location:
                return True
        return False

    def destroyed(self):
        for coords in self.coordinates:
            if board_display[coords['row']][coords['col']] == 'O':
                return False
            elif board_display[coords['row']][coords['col']] == '*':
                raise RuntimeError("Board display inaccurate")
        return True
    

In [14]:
#Settings Variables
row_size = 4 #number of rows
col_size = 4 #number of columns
n_total= int(np.floor(np.log2(col_size*row_size)+1))
num_ships = 2
max_ship_size = 3
min_ship_size = 2
num_turns = 10

#Create lists
ship_list = []

board = [[0] * col_size for x in range(row_size)]

board_display = [["O"] * col_size for x in range(row_size)]




In [15]:
#Functions
def print_board(board_array):
    print("\n  " + " ".join(str(x) for x in range(1, col_size + 1)))
    for r in range(row_size):
        print(str(r + 1) + " " + " ".join(str(c) for c in board_array[r]))
    print()

def search_locations(size, orientation):
    locations = []

    if orientation != 'horizontal' and orientation != 'vertical':
        raise ValueError("Orientation must have a value of either 'horizontal' or 'vertical'.")

    if orientation == 'horizontal':
        if size <= col_size:
            for r in range(row_size):
                for c in range(col_size - size + 1):
                    if 1 not in board[r][c:c+size]:
                        locations.append({'row': r, 'col': c})
    elif orientation == 'vertical':
        if size <= row_size:
            for c in range(col_size):
                for r in range(row_size - size + 1):
                    if 1 not in [board[i][c] for i in range(r, r+size)]:
                        locations.append({'row': r, 'col': c})

    if not locations:
        return 'None'
    else:
        return locations

def random_location():
    size = randint(min_ship_size, max_ship_size)
    orientation = 'horizontal' if randint(0, 1) == 0 else 'vertical'

    locations = search_locations(size, orientation)
    if locations == 'None':
        return 'None'
    else:
        return {'location': locations[randint(0, len(locations) - 1)], 'size': size, \
                'orientation': orientation}

def get_row():
    while True:
        try:
            guess = int(input("Row Guess: "))
            if guess in range(1, row_size + 1):
                return guess - 1
            else:
                print("\nOops, that's not even in the ocean.")
        except ValueError:
            print("\nPlease enter a number")

def get_col():
    while True:
        try:
            guess = int(input("Column Guess: "))
            if guess in range(1, col_size + 1):
                return guess - 1
            else:
                print("\nOops, that's not even in the ocean.")
        except ValueError:
            print("\nPlease enter a number")

# Create the ships
temp = 0
while temp < num_ships:
    ship_info = random_location()
    if ship_info == 'None':
        continue
    else:
        ship_list.append(Ship(ship_info['size'], ship_info['orientation'], ship_info['location']))
        temp += 1
del temp
def merged_grid(pl):
    """Return board with disp-overlay (X, *, D)."""
    merged = [row[:] for row in pl['board']]          # copy ships/sea
    for r in range(row_size):
        for c in range(col_size):
            if pl['disp'][r][c] in ('X', '*', 'D'):   # only 3 dynamic marks
                merged[r][c] = pl['disp'][r][c]
    return merged

def repaint_own(pl):
    repaint(pl['own'], merged_grid(pl), own_colours)

def repaint_all():
    for pl in players:
        repaint_own(pl)                          # own board
        repaint(pl['atk'], pl['view'], view_colours)  # shot history


### Quantum gate

In [16]:
from math      import ceil, log2
# def shot(a1_row, a1_col, theta1,
#          a2_row, a2_col, theta2,
#          def_row, def_col, theta3,
#          atk1_row, atk1_col, theta4,
#          atk2_row=-1, atk2_col=-1, theta5=0.0,
#          defended=False):
#     global n_total, row_size
#     
#     coords = [(a1_row-1) * row_size +a1_col,(a2_row-1)*row_size+a2_col,(def_row-1)*row_size +def_col,  (atk_row-1)*row_size+ atk_col]
#     phases = [theta1, theta2, theta3, theta4]
#     fields = [1, 1, int(defended), 0]
#     
#     states = len(coords)  # number of the states
def shot(a1_row, a1_col, theta1,
         a2_row, a2_col, theta2,
         def_row, def_col, theta3,
         atk1_row, atk1_col, theta4,
         atk2_row=-1, atk2_col=-1, theta5=0.0,
         defended=False):
    """
    Build and run the “four-event” quantum circuit used by the Battleship GUI.

    Any coordinate passed as –1 is ignored.  
    `defended` tells the circuit whether the defender actually placed a shield.
    Returns two integers (A, B) that the GUI converts back to board squares.
    """
    global n_total, row_size, col_size        # defined in your main script

    # ───────────────────────── 1. gather valid events ──────────────────────────
    coords, phases, fields = [], [], []      # field 0 → register A, 1 → B

    # starter’s two shots
    coords += [a1_row*col_size + a1_col+1,  a2_row*col_size + a2_col+1]
    phases += [theta1*np.pi, theta2*np.pi]
    fields += [0, 0]                         # act on qReg_A

    # optional defence
    if defended and def_row >= 0 and def_col >= 0:
        coords.append(def_row*col_size + def_col+1)
        phases.append(theta3*np.pi)
        fields.append(1)                    # act on qReg_B

    # responder’s first (always present) attack
    coords.append(atk1_row*col_size + atk1_col+1)
    phases.append(theta4*np.pi)
    fields.append(1)                        # act on qReg_B

    # optional second attack
    if atk2_row >= 0 and atk2_col >= 0:
        coords.append(atk2_row*col_size + atk2_col+1)
        phases.append(theta5*np.pi)
        fields.append(1)

    # pad lists to a power of two so the ancilla loop works
    states = len(coords)
    n_anc   = ceil(log2(states))
    pad_len = 2**n_anc - states
    coords += [0]*pad_len
    phases += [0.0]*pad_len
    fields += [0]*pad_len
    n = n_total

    def UGate(circuit, num, phase, qReg, controls):
        #qReg = QuantumRegister(n)
        gate = QuantumCircuit(qReg, name='$U_{'+str(num)+'}$')
        num_bin = bin(num)[2:].zfill(n)
        for i in range(n):
            if num_bin[-i-1] == '1':
                #gate.x(i)
                qubits = [controls[j] for j in range(controls.size)] + [qReg[i]]
                x = XGate().control(controls.size)
                circuit.append(x, qubits)
        GP = GlobalPhaseGate(phase)
        GP.num_qubits = 1
        qubits = [controls[j] for j in range(controls.size)] + [qReg[0]]
        circuit.append(GP.control(controls.size), qubits)
        return circuit

    N_ancilas = int(np.ceil(np.log2(states)))

    ancila_qReg = QuantumRegister(N_ancilas, name='Qancila')
    qReg_A = QuantumRegister(n, name='qA')
    qReg_B = QuantumRegister(n, name='qB')
    ancila_cReg = ClassicalRegister(N_ancilas, name='Cancila')
    cReg_A = ClassicalRegister(n, name='cA')
    cReg_B = ClassicalRegister(n, name='cB')

    circuit = QuantumCircuit(ancila_qReg, qReg_A, qReg_B, ancila_cReg, cReg_A, cReg_B)

    for i in range(N_ancilas):
        circuit.h(ancila_qReg[i])
    circuit.barrier()
    for i in range(2**N_ancilas):
        bin_num = bin(i)[2:].zfill(N_ancilas)
        #  Putting X gates
        for j in range(N_ancilas):
            if bin_num[j] == '1':
                circuit.x(ancila_qReg[-j-1])

        if fields[i] == 0:
            circuit = UGate(circuit, coords[i], phases[i], qReg_A, ancila_qReg)
        else:
            circuit = UGate(circuit, coords[i], phases[i], qReg_B, ancila_qReg)

        #  Removing the X gates
        for j in range(N_ancilas):
            if bin_num[j] == '1':
                circuit.x(ancila_qReg[-j-1])
        circuit.barrier()

    for i in range(N_ancilas):
        circuit.h(ancila_qReg[i])
    circuit.barrier()
    circuit.measure(ancila_qReg, ancila_cReg)
    circuit.measure(qReg_A, cReg_A)
    circuit.measure(qReg_B, cReg_B)

    # simulator = AerSimulator(shots=1)
    # compiled_circuit = transpile(circuit, simulator)
    # ancila_values = '11'
    # while ancila_values != '00':
    #     result = simulator.run(compiled_circuit).result()
    #     counts = result.get_counts()
    #     ancila_values = list(counts.keys())[0][-2:]
    # 
    # key = list(counts.keys())[0]
    # B = int(key[:4], 2)
    # A = int(key[5:9], 2)
    # ----------------- post-select on ancilla = |0…0> -----------------
    sim     = AerSimulator(shots=1)
    circ    = transpile(circuit, sim)
    zeroTag = '0' * n_anc

    while True:
        result = sim.run(circ).result()
        counts = result.get_counts()
        key    = next(iter(counts))          # single shot ⇒ one key like '0000 0000 00'
        # split classical registers (Qiskit inserts spaces between them)
        bits_B, bits_A, bits_anc = key.split(' ')
        if bits_anc == zeroTag:              # we keep only runs where ancilla is |0…0>
            break

    A = int(bits_A, 2)
    B = int(bits_B, 2)
    return A, B
    




In [ ]:
import tkinter as tk
from tkinter import simpledialog, messagebox

# ───────── helpers reused from last version ──────────────────────────
CELL = 36
own_colours = {
    0:"#d0e7ff",          # empty sea
    1:"#4f9cff",          # your ship
    'O':"#d0e7ff",        # undiscovered sea (same blue as 0)
    'X':"#ff4d4d",        # a hit
    '*':"#ffffff",        # a miss
    'D':"#66ff66",        # defender’s shield
}
view_colours = {'O':"#808080", 'X':"#ff4d4d", '*':"#ffffff",
                'P':"#bbbbbb", 'D':"#66ff66"}

# ───────── NEW helper functions (Fix ①) ─────────────────────────────
def merged_own_grid(pl):
    """board (0/1) overlaid with X * D marks from disp layer."""
    g = [row[:] for row in pl['board']]
    for r in range(row_size):
        for c in range(col_size):
            if pl['disp'][r][c] in ('X', '*', 'D'):
                g[r][c] = pl['disp'][r][c]
    return g

def draw_own(pl):
    repaint(pl['own'], merged_own_grid(pl), own_colours)

def idx_to_rc(idx):
    """Convert circuit index → (row, col).
       0  ➜  (-1,-1)   means “no hit”
       1  ➜  (0,0)     first tile
       .
    """
    if idx == 0:
        return -1, -1                 # sentinel = “nothing happened”
    idx -= 1                           # shift from 1-based to 0-based
    return idx // col_size, idx % col_size
# ────────────────────────────────────────────────────────────────────

def repaint(cv, grid, pal):
    cv.delete("all")
    for r in range(row_size):
        for c in range(col_size):
            x0, y0 = c*CELL, r*CELL
            cv.create_rectangle(x0, y0, x0+CELL, y0+CELL,
                                fill=pal[grid[r][c]], outline="black")

def hit(defender, r, c):
    for s in defender['ships']:
        if s.contains({'row': r, 'col': c}):
            defender['disp'][r][c] = 'X'
            if s.destroyed():
                defender['ships'].remove(s)
            return
    defender['disp'][r][c] = '*'

def build_fleet():
    global board, board_display
    board, board_display = (
        [[0]*col_size for _ in range(row_size)],
        [["O"]*col_size for _ in range(row_size)]
    )
    ships, placed = [], 0
    while placed < num_ships:
        info = random_location()
        if info == 'None':
            continue
        ship = Ship(info['size'], info['orientation'], info['location'])
        ships.append(ship)

        # ─── NEW: burn the ship’s squares into the board ───────────
        for sq in ship.coordinates:              # ← use .coordinates
            board[sq['row']][sq['col']] = 1  # 1 = my ship (dark blue)
        # ───────────────────────────────────────────────────────────

        placed += 1

    return {
        'board': board,
        'disp':  board_display,
        'ships': ships,
        'view':  [["O"]*col_size for _ in range(row_size)]
    }


# ───────── build two windows ─────────────────────────────────────────
p1, p2 = build_fleet(), build_fleet()
players = [p1, p2]

root = tk.Tk()
root.withdraw()
for idx, pl in enumerate(players, start=1):
    w = tk.Toplevel()
    w.title(str(idx))                        # “1” / “2”
    tk.Label(w, text="your fleet").grid(row=0, column=0)
    tk.Label(w, text="your shots").grid(row=0, column=1)

    own = tk.Canvas(w, width=col_size*CELL, height=row_size*CELL)
    atk = tk.Canvas(w, width=col_size*CELL, height=row_size*CELL)
    own.grid(row=1, column=0)
    atk.grid(row=1, column=1, padx=6)

    pl.update(win=w, own=own, atk=atk)
    draw_own(pl)                                # ← Fix ② (was repaint(...))
    repaint(atk, pl['view'], view_colours)

# ───────── round-state variables ─────────────────────────────────────
starter = 0                 # 0 or 1  (flips after each quantum step)
phase   = "A1"              # A1→A2→R_DEF/skip→R_ATK
data    = {}                # all coords + angles for current round

def prompt_angle(label, win):
    val = simpledialog.askfloat("secret", f"{label} (0–2π)", parent=win)
    return val if val is not None else prompt_angle(label, win)

def finish_round():
    global phase, starter, data
    # fill missing keys so quantum_gate always gets all 16 params
    data.setdefault('def_row', -1)
    data.setdefault('def_col', -1)
    data.setdefault('theta3', 0.0)
    data.setdefault('atk2_row', -1)
    data.setdefault('atk2_col', -1)
    data.setdefault('theta5', 0.0)

    # call your circuit
    s1, s2 = shot(**data)

    # ← Fix ③: simple, correct conversion
    h1r, h1c = idx_to_rc(s1)     # hit on responder
    h2r, h2c = idx_to_rc(s2)     # hit on starter

    # apply hits
    hit(players[1-starter], h1r, h1c)
    hit(players[starter],   h2r, h2c)

    # update views
    players[starter]['view'][h1r][h1c]     = players[1-starter]['disp'][h1r][h1c]
    players[1-starter]['view'][h2r][h2c]   = players[starter]['disp'][h2r][h2c]

    # update views of the two returned squares  (already in your code)
    players[starter]['view'][h1r][h1c]   = players[1-starter]['disp'][h1r][h1c]
    players[1-starter]['view'][h2r][h2c] = players[starter]['disp'][h2r][h2c]

    # ─── NEW: clear all other 'P' so they can be picked next round ───
    for pl in players:
        for r in range(row_size):
            for c in range(col_size):
                if pl['view'][r][c] == 'P':
                    pl['view'][r][c] = 'O'
    # ────────────────────────────────────────────────────────────────

    # redraw
    for pl in players:
        draw_own(pl)
        repaint(pl['atk'], pl['view'], view_colours)
    # win?
    if not players[0]['ships'] or not players[1]['ships']:
        winner = "1" if players[1]['ships'] == [] else "2"
        messagebox.showinfo("game over", f"Player {winner} wins!")
        for pl in players:
            pl['win'].destroy()
        return

    # next round
    data.clear()
    starter = 1 - starter
    phase   = "A1"
    messagebox.showinfo("next round", f"New round — Player {starter+1} starts")
    defaults = dict(
        def_row=-1, def_col=-1, theta3=0.0,
        atk1_row=-1, atk1_col=-1, theta4=0.0,
        atk2_row=-1, atk2_col=-1, theta5=0.0,
        defended=False
    )
    for k, v in defaults.items():
        data.setdefault(k, v)

# ───────── NEW mouse handler ────────────────────────────────────────
def click(event):
    global phase, starter, data, row_size, col_size

    who  = 0 if event.widget in (p1['own'], p1['atk']) else 1
    me   = players[who]
    opp  = players[1-who]
    col, row = event.x // CELL, event.y // CELL
    if not (0 <= row < row_size and 0 <= col < col_size):
        return

    # (all your original state-machine logic is unchanged …)
    # ── starter attacks ────────────────────────────────────────────
    if phase == "A1" and who == starter and event.widget is me['atk']:
        if me['view'][row][col] in ('X', '*', 'P'):
            return
        θ1 = prompt_angle("θ₁", me['win'])
        data.update(a1_row=row, a1_col=col, theta1=θ1)
        me['view'][row][col] = 'P'
        repaint(me['atk'], me['view'], view_colours)
        phase = "A2"
        return

    if phase == "A2" and who == starter and event.widget is me['atk']:
        if me['view'][row][col] in ('X', '*', 'P'):
            return
        θ2 = prompt_angle("θ₂", me['win'])
        data.update(a2_row=row, a2_col=col, theta2=θ2)
        me['view'][row][col] = 'P'
        repaint(me['atk'], me['view'], view_colours)
        phase = "R_CHOICE"
        messagebox.showinfo(
            "responder",
            f"Player {2-starter}: choose (a) a defence tile on your board *or* "
            "directly an attack tile on opponent board"
        )
        return

    # ── responder’s first decision ─────────────────────────────────
    responder = 1 - starter
    if phase == "R_CHOICE" and who == responder:
        # ① chose to DEFEND
        if event.widget is me['own']:
            θ3 = prompt_angle("θ₃", me['win'])
            data.update(def_row=row, def_col=col, theta3=θ3, defended=True)
            me['disp'][row][col] = 'D'
            draw_own(me)                       # ← Fix ②
            phase = "R_ATK1"
            messagebox.showinfo(
                "attack",
                f"Player {who+1}: now pick ONE attack tile on opponent board"
            )
            return
        # ① chose first ATTACK  (double-attack route)
        if event.widget is me['atk']:
            if me['view'][row][col] in ('X', '*', 'P'):
                return
            θ4 = prompt_angle("θ₄", me['win'])
            data.update(atk1_row=row, atk1_col=col, theta4=θ4, defended=False)
            me['view'][row][col] = 'P'
            repaint(me['atk'], me['view'], view_colours)
            phase = "R_ATK2"
            messagebox.showinfo("second attack",
                                f"Player {who+1}: pick SECOND attack tile on opponent board")
            return

    # ── responder’s single attack when defence was chosen ──────────
    if phase == "R_ATK1" and who == responder and event.widget is me['atk']:
        if me['view'][row][col] in ('X', '*', 'P'):
            return
        θ4 = prompt_angle("θ₄", me['win'])
        data.update(atk1_row=row, atk1_col=col, theta4=θ4,
                    atk2_row=-1, atk2_col=-1, theta5=0.0)
        me['view'][row][col] = 'P'
        repaint(me['atk'], me['view'], view_colours)
        finish_round()
        return

    # ── responder’s SECOND attack when no defence ──────────────────
    if phase == "R_ATK2" and who == responder and event.widget is me['atk']:
        if me['view'][row][col] in ('X', '*', 'P'):
            return
        θ5 = prompt_angle("θ₅", me['win'])
        data.update(atk2_row=row, atk2_col=col, theta5=θ5)
        me['view'][row][col] = 'P'
        repaint(me['atk'], me['view'], view_colours)
        finish_round()
        return

# bind all canvases
for pl in players:
    pl['own'].bind("<Button-1>", click)
    pl['atk'].bind("<Button-1>", click)

root.mainloop()


In [7]:
import random
import tkinter as tk
from tkinter import simpledialog, messagebox

# ───────── parameters you had as globals elsewhere ──────────────────
# row_size   = 6                  # board = 6 × 6  (change as you like)
# col_size   = 6
# num_ships  = 3
CELL       = 36

# ───────── stubs that let the file run standalone ───────────────────
class Ship:
    """Fake ship object just big enough for the GUI test."""
    def __init__(self, size, orient, loc):
        self.coords = {(loc['row'] + (i if orient == 'V' else 0),
                        loc['col'] + (i if orient == 'H' else 0))
                       for i in range(size)}

    def contains(self, rc):
        return (rc['row'], rc['col']) in self.coords

    def destroyed(self):
        # always False until you wire hit-counting
        return False

def random_location():
    """Return a stub ship-placement dict."""
    return dict(size=2,
                orientation=random.choice(['H', 'V']),
                location=dict(row=random.randrange(row_size),
                              col=random.randrange(col_size)))

def shot(**kwargs):
    """Fake quantum circuit: just fire at the defender’s first attack tile."""
    flat = kwargs['atk1_row'] * col_size + kwargs['atk1_col']
    return flat, -1             # (hit on responder, no counter-hit)

# ───────── palettes ─────────────────────────────────────────────────
own_colours  = {0:"#d0e7ff", 1:"#4f9cff", 'X':"#ff4d4d",
                '*':"#ffffff", 'D':"#66ff66"}
view_colours = {'O':"#808080", 'X':"#ff4d4d", '*':"#ffffff",
                'P':"#bbbbbb", 'D':"#66ff66"}

# ───────── helpers ─────────────────────────────────────────────────
def repaint(cv, grid, pal):
    cv.delete("all")
    for r in range(row_size):
        for c in range(col_size):
            x0, y0 = c * CELL, r * CELL
            cv.create_rectangle(x0, y0, x0+CELL, y0+CELL,
                                fill=pal[grid[r][c]], outline="black")

def merged_grid(pl):
    g = [row[:] for row in pl['board']]
    for r in range(row_size):
        for c in range(col_size):
            if pl['disp'][r][c] in ('X', '*', 'D'):
                g[r][c] = pl['disp'][r][c]
    return g

def repaint_own(pl):
    repaint(pl['own'], merged_grid(pl), own_colours)

def repaint_all():
    for pl in players:
        repaint_own(pl)
        repaint(pl['atk'], pl['view'], view_colours)

def idx_to_rc(idx):
    return (-1, -1) if idx < 0 else divmod(idx, col_size)

def hit(defender, r, c):
    for s in defender['ships']:
        if s.contains({'row': r, 'col': c}):
            defender['disp'][r][c] = 'X'
            return
    defender['disp'][r][c] = '*'

# ───────── fleet factory ────────────────────────────────────────────
def build_fleet():
    board  = [[0]*col_size for _ in range(row_size)]
    disp   = [['O']*col_size for _ in range(row_size)]
    ships  = []
    while len(ships) < num_ships:
        info = random_location()
        ships.append(Ship(info['size'], info['orientation'], info['location']))
    return dict(board=board, disp=disp, ships=ships,
                view=[['O']*col_size for _ in range(row_size)])

# ───────── create windows *before* anything touches 'own' ───────────
p1, p2  = build_fleet(), build_fleet()
players = [p1, p2]

root = tk.Tk(); root.withdraw()
for idx, pl in enumerate(players, start=1):
    w = tk.Toplevel()
    w.title(f"Player {idx}")
    tk.Label(w, text="your fleet").grid(row=0, column=0)
    tk.Label(w, text="your shots").grid(row=0, column=1)

    own = tk.Canvas(w, width=col_size*CELL, height=row_size*CELL)
    atk = tk.Canvas(w, width=col_size*CELL, height=row_size*CELL)
    own.grid(row=1, column=0); atk.grid(row=1, column=1, padx=6)

    pl.update(win=w, own=own, atk=atk)

# first full paint (now 'own' exists)
repaint_all()

# ───────── round-state ------------------------------------------------
starter = 0          # player index who starts the round
phase   = "A1"
data    = dict(defended=False)

# ───────── prompt helper ---------------------------------------------
def prompt_angle(label, win):
    while True:
        val = simpledialog.askfloat("secret", f"{label} (0–2π)", parent=win)
        if val is not None:
            return val

# ───────── end-of-round routine --------------------------------------
def finish_round():
    global starter, phase, data

    # circuit call
    s1, s2 = shot(**data)
    # convert circuit output
    # convert circuit output (already there)
    h1r, h1c = idx_to_rc(s1)
    h2r, h2c = idx_to_rc(s2)
    
    # ─── NEW: accept only squares that were actually picked ───────────
    starter_choices   = {(data['a1_row'],  data['a1_col']),
                         (data['a2_row'],  data['a2_col'])}
    
    responder_choices = {(data['atk1_row'], data['atk1_col'])}
    if data.get('atk2_row', -1) >= 0:
        responder_choices.add((data['atk2_row'], data['atk2_col']))
    
    if (h1r, h1c) not in starter_choices:
        h1r = h1c = -1                # ignore stray index
    if (h2r, h2c) not in responder_choices:
        h2r = h2c = -1
    # ──────────────────────────────────────────────────────────────────
    
    # apply hits (only if not –1, as you already guard)
    if h1r != -1:
        hit(players[1 - starter], h1r, h1c)
        players[starter]['view'][h1r][h1c] = players[1 - starter]['disp'][h1r][h1c]
    
    if h2r != -1:
        hit(players[starter], h2r, h2c)
        players[1 - starter]['view'][h2r][h2c] = players[starter]['disp'][h2r][h2c]


    # new round or win
    if not players[0]['ships'] or not players[1]['ships']:
        winner = "1" if players[1]['ships'] else "2"
        messagebox.showinfo("game over", f"Player {winner} wins!")
        for pl in players: pl['win'].destroy()
        root.destroy()
        return

    data.clear()
    phase   = "A1"
    starter = 1 - starter
    messagebox.showinfo("next round", f"New round — Player {starter+1} starts")

# ───────── click handler (starter’s first shot only, for demo) -------
def click(event):
    global phase, data

    who = 0 if event.widget in (p1['own'], p1['atk']) else 1
    me  = players[who]

    if phase != "A1" or who != starter or event.widget is not me['atk']:
        return                           # ignore everything but the very first shot

    col = event.x // CELL
    row = event.y // CELL
    if me['view'][row][col] != 'O':
        return

    θ = prompt_angle("θ₁", me['win'])
    data.update(a1_row=row, a1_col=col, theta1=θ,
                atk1_row=row, atk1_col=col, theta4=θ)  # feed the stub circuit

    me['view'][row][col] = 'P'
    repaint(me['atk'], me['view'], view_colours)

    phase = "DONE"
    finish_round()

# ───────── bind after canvases exist ---------------------------------
for pl in players:
    pl['own'].bind("<Button-1>", click)
    pl['atk'].bind("<Button-1>", click)

root.mainloop()


KeyboardInterrupt: 

In [7]:
import tkinter as tk
from tkinter import simpledialog, messagebox

# ───────── helpers reused from last version ──────────────────────────
CELL = 36
own_colours = {
    0:"#d0e7ff",          # empty sea
    1:"#4f9cff",          # your ship
    'O':"#d0e7ff",        # undiscovered sea (same blue as 0)
    'X':"#ff4d4d",        # a hit
    '*':"#ffffff",        # a miss
    'D':"#66ff66",        # defender’s shield
}
view_colours = {'O':"#808080", 'X':"#ff4d4d", '*':"#ffffff",
                'P':"#bbbbbb", 'D':"#66ff66"}
# ───── add just after your colour dictionaries ──────────────────────
def merged_own_grid(pl):
    """board (0/1) overlaid with dynamic marks X * D."""
    g = [row[:] for row in pl['board']]          # shallow copy
    for r in range(row_size):
        for c in range(col_size):
            if pl['disp'][r][c] in ('X', '*', 'D'):
                g[r][c] = pl['disp'][r][c]
    return g

def draw_own(pl):                                # single point of truth
    repaint(pl['own'], merged_own_grid(pl), own_colours)

def idx_to_rc(idx):                              # correct decoder
    return (-1, -1) if idx < 0 else (idx // col_size, idx % col_size)
# ────────────────────────────────────────────────────────────────────


def repaint(cv, grid, pal):
    cv.delete("all")
    for r in range(row_size):
        for c in range(col_size):
            x0,y0 = c*CELL, r*CELL
            cv.create_rectangle(x0,y0,x0+CELL,y0+CELL,
                                fill=pal[grid[r][c]], outline="black")

def hit(defender, r, c):
    for s in defender['ships']:
        if s.contains({'row':r,'col':c}):
            defender['disp'][r][c] = 'X'
            if s.destroyed(): defender['ships'].remove(s)
            return
    defender['disp'][r][c] = '*'

def build_fleet():
    global board, board_display
    board, board_display = [[0]*col_size for _ in range(row_size)], [["O"]*col_size for _ in range(row_size)]
    ships, placed = [], 0
    while placed < num_ships:
        info = random_location()
        if info == 'None': continue
        ships.append(Ship(info['size'], info['orientation'], info['location']))
        placed += 1
    return {'board':board, 'disp':board_display, 'ships':ships,
            'view':[["O"]*col_size for _ in range(row_size)]}

# ───────── build two windows ─────────────────────────────────────────
p1, p2 = build_fleet(), build_fleet()
players = [p1, p2]

root = tk.Tk(); root.withdraw()
for idx, pl in enumerate(players, start=1):
    w = tk.Toplevel()
    w.title(str(idx))                                     # “1” / “2”
    tk.Label(w,text="your fleet").grid(row=0,column=0)
    tk.Label(w,text="your shots").grid(row=0,column=1)

    own = tk.Canvas(w,width=col_size*CELL,height=row_size*CELL)
    atk = tk.Canvas(w,width=col_size*CELL,height=row_size*CELL)
    own.grid(row=1,column=0); atk.grid(row=1,column=1,padx=6)

    pl.update(win=w, own=own, atk=atk)
    repaint(own, pl['board'], own_colours)
    repaint(atk, pl['view'],  view_colours)
    
    for pl in players:
        repaint_own(pl)                              # ← uses merged layer
    repaint(pl['atk'], pl['view'], view_colours)


# ───────── round-state variables ─────────────────────────────────────
starter = 0                 # 0 or 1  (flips after each quantum step)
phase   = "A1"              # A1→A2→R_DEF/skip→R_ATK
data    = {}                # all coords + angles for current round

def prompt_angle(label, win):
    val = simpledialog.askfloat("secret", f"{label} (0–2π)", parent=win)
    return val if val is not None else prompt_angle(label, win)
def idx_to_rc(idx):
    """Turn a linear index (0 … row_size*col_size-1)
       into (row, col).  If idx is –1 return (-1,-1)."""
    if idx < 0:
        return -1, -1
    return divmod(idx, col_size)    # (row, col)

def finish_round():
    global phase, starter, data
    # fill missing keys so quantum_gate always gets all 16 params
    data.setdefault('def_row', -1)
    data.setdefault('def_col', -1)
    data.setdefault('theta3' , 0.0)
    data.setdefault('atk2_row', -1)
    data.setdefault('atk2_col', -1)
    data.setdefault('theta5' , 0.0)

    # call your circuit
    s1, s2 = shot(**data)
    # if s1==0:
    #     h1r,h1c, h2r,h2c = 0,0,s2//row_size,s2%row_size-1
    # elif s2==0:
    #     h1r,h1c, h2r,h2c = s1//row_size, s1%row_size-1,0,0
    # else:
    #     h1r,h1c, h2r,h2c = s1//row_size, s1%row_size-1,s2//row_size,s2%row_size-1
    # convert the two positions returned by the circuit
    h1r, h1c = idx_to_rc(s1)        # hit on responder
    h2r, h2c = idx_to_rc(s2)        # counter-hit on starter
    if s2 == -1:
        h2r, h2c = -1, -1




    # apply hits
    if h1r != -1:
        hit(players[1 - starter], h1r, h1c)
    if h2r != -1:
        hit(players[starter], h2r, h2c)

    # hit(players[1-starter], h1r,h1c)    # responder’s board
    # hit(players[starter]  , h2r,h2c)    # starter’s board
    # # update views
    if h1r != -1:
        players[starter]['view'][h1r][h1c] = players[1 - starter]['disp'][h1r][h1c]
    if h2r != -1:
        players[1 - starter]['view'][h2r][h2c] = players[starter]['disp'][h2r][h2c]

    # players[starter]['view'][h1r][h1c]       = players[1-starter]['disp'][h1r][h1c]
    # players[1-starter]['view'][h2r][h2c]     = players[starter]['disp'][h2r][h2c]
    # redraw
    for pl in players:
        repaint_own(pl)
        repaint(pl['atk'], pl['view'], view_colours)

    # win?
    if not players[0]['ships'] or not players[1]['ships']:
        winner = "1" if players[1]['ships']==[] else "2"
        messagebox.showinfo("game over", f"Player {winner} wins!")
        for pl in players: pl['win'].destroy()
        return

    # next round
    data.clear()
    starter = 1 - starter
    phase   = "A1"
    messagebox.showinfo("next round", f"New round — Player {starter+1} starts")
    defaults = dict(
        def_row=-1, def_col=-1, theta3=0.0,
        atk1_row=-1, atk1_col=-1, theta4=0.0,
        atk2_row=-1, atk2_col=-1, theta5=0.0,
        defended=False
    )
    for k,v in defaults.items():
        data.setdefault(k, v)

# ───────── NEW mouse handler ────────────────────────────────────────
def click(event):
    global phase, starter, data, row_size, col_size

    who  = 0 if event.widget in (p1['own'], p1['atk']) else 1
    me   = players[who]
    opp  = players[1-who]
    col, row = event.x // CELL, event.y // CELL
    if not (0 <= row < row_size and 0 <= col < col_size):
        return

    # ── starter attacks ────────────────────────────────────────────
    if phase == "A1" and who == starter and event.widget is me['atk']:
        if me['view'][row][col] in ('X','*','P'): return
        θ1 = prompt_angle("θ₁", me['win'])
        data.update(a1_row=row, a1_col=col, theta1=θ1)
        me['view'][row][col] = 'P'; repaint(me['atk'], me['view'], view_colours)
        phase = "A2"; return

    if phase == "A2" and who == starter and event.widget is me['atk']:
        if me['view'][row][col] in ('X','*','P'): return
        θ2 = prompt_angle("θ₂", me['win'])
        data.update(a2_row=row, a2_col=col, theta2=θ2)
        me['view'][row][col] = 'P'; repaint(me['atk'], me['view'], view_colours)
        phase = "R_CHOICE"
        messagebox.showinfo(
            "responder",
            f"Player {2-starter}: choose (a) a defence tile on your board *or* "
            "directly an attack tile on opponent board"
        )
        return

    # ── responder’s first decision ─────────────────────────────────
    responder = 1 - starter
    if phase == "R_CHOICE" and who == responder:
        # ① chose to DEFEND
        if event.widget is me['own']:
            θ3 = prompt_angle("θ₃", me['win'])
            data.update(def_row=row, def_col=col, theta3=θ3, defended=True)
            me['disp'][row][col] = 'D'; repaint(me['own'], me['disp'], own_colours)
            phase = "R_ATK1"
            messagebox.showinfo(
                "attack",
                f"Player {who+1}: now pick ONE attack tile on opponent board"
            )
            return
        # ① chose first ATTACK  (double-attack route)
        if event.widget is me['atk']:
            if me['view'][row][col] in ('X','*','P'): return
            θ4 = prompt_angle("θ₄", me['win'])
            data.update(atk1_row=row, atk1_col=col, theta4=θ4,
                        defended=False)        # no defence
            me['view'][row][col] = 'P'; repaint(me['atk'], me['view'], view_colours)
            phase = "R_ATK2"
            messagebox.showinfo("second attack",
                                f"Player {who+1}: pick SECOND attack tile on opponent board")
            return

    # ── responder’s single attack when defence was chosen ──────────
    if phase == "R_ATK1" and who == responder and event.widget is me['atk']:
        if me['view'][row][col] in ('X','*','P'): return
        θ4 = prompt_angle("θ₄", me['win'])
        data.update(atk1_row=row, atk1_col=col, theta4=θ4,
                    atk2_row=-1, atk2_col=-1, theta5=0.0)
        me['view'][row][col] = 'P'; repaint(me['atk'], me['view'], view_colours)
        finish_round()                       # ← calls quantum_gate & resets
        return

    # ── responder’s SECOND attack when no defence ──────────────────
    if phase == "R_ATK2" and who == responder and event.widget is me['atk']:
        if me['view'][row][col] in ('X','*','P'): return
        θ5 = prompt_angle("θ₅", me['win'])
        data.update(atk2_row=row, atk2_col=col, theta5=θ5)
        me['view'][row][col] = 'P'; repaint(me['atk'], me['view'], view_colours)
        finish_round()
        return
        # ─── call quantum circuit ───────────────────────────────────
        s1, s2 = shot(**data)
        if s1==0:
            h1r,h1c, h2r,h2c = 0,0,s2//row_size,s2%row_size-1
        elif s2==0:
            h1r,h1c, h2r,h2c = 1//row_size, s1%row_size-1,0,0
        else:
            h1r,h1c, h2r,h2c = 1//row_size, s1%row_size-1,s2//row_size,s2%row_size-1
        print( h1r,h1c, h2r,h2c)


    # apply hits
        hit(players[responder], hit_r , hit_c )   # on responder’s board
        hit(players[starter  ], hit_r2, hit_c2)   # on starter’s board
        # update views
        players[starter]['view'][hit_r ][hit_c ] = players[responder]['disp'][hit_r ][hit_c ]
        players[responder]['view'][hit_r2][hit_c2] = players[starter  ]['disp'][hit_r2][hit_c2]

        # redraw all
        for pl in players:
            repaint_own(pl)
            repaint(pl['atk'], pl['view'], view_colours)

        # win check
        if not players[0]['ships'] or not players[1]['ships']:
            winner = "1" if players[1]['ships']==[] else "2"
            messagebox.showinfo("game over", f"Player {winner} wins!")
            for pl in players: pl['win'].destroy()
            return
        repaint_all()
        # set up next round
        data.clear()
        phase="A1"
        starter = 1-starter
        messagebox.showinfo("next round", f"New round — Player {starter+1} starts")
        return

# bind all canvases
for pl in players:
    pl['own'].bind("<Button-1>", click)
    pl['atk'].bind("<Button-1>", click)

root.mainloop()

KeyError: 'own'